In [1]:
!pip install langchain langchain-community openai tiktoken

In [3]:
import os
import numpy as np
from numpy import dot
from numpy.linalg import norm
import pandas as pd
from langchain.embeddings import OpenAIEmbeddings
os.environ['OPENAI_API_KEY'] = '여러분들의 OpenAI Key 값'

In [4]:
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")
query_result = embeddings.embed_query('저는 배가 고파요')
print(query_result)

/var/folders/hl/t9j0xf754r12rg2z1pd0sg_80000gn/T/ipykernel_24463/2858411033.py:1: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")


[-0.016637360179694066, -0.021788898203996467, 0.015218060008672592, -0.027229550411783026, -0.036822973580909454, 0.011774940995488552, -0.034562605331125006, -0.006715396218388499, -0.023996698883951003, -0.016821344811453674, -0.008134697320732518, 0.010822169159143375, -0.010559335966606536, -0.022669389166164237, 0.011308411729489709, -0.0049346991270603725, 0.012070628453507813, -0.0029240226430165567, 0.008134697320732518, -0.016137977579535345, 0.0013757691074475543, -0.014547834669211744, 0.01825378780625517, -0.012701429046918771, 0.0032525645993488777, 0.006432850187165443, 0.005342090808323108, -0.019016004530273273, -0.00918603102220242, -0.0017347011662188622, 0.03451003776129509, -0.01644023644944462, 0.00011386029314690746, 0.003149073942476009, 0.007372479665391869, -0.00548007881648154, -0.006354000229404392, 0.003472687805552094, -0.003784802396312567, -0.0022505118304413396, -0.02243283882721981, -0.010362211305034544, 0.011354407887429611, -0.02473263368569924, 0.0

In [5]:
data = ['저는 배가 고파요',
        '저기 배가 지나가네요',
        '굶어서 허기가 지네요',
        '허기 워기라는 게임이 있는데 즐거워',
        '스팀에서 재밌는 거 해야지',
        '스팀에어프라이어로 연어구이 해먹을거야']

df = pd.DataFrame(data, columns=['text'])
df

,text
0,저는 배가 고파요
1,저기 배가 지나가네요
2,굶어서 허기가 지네요
3,허기 워기라는 게임이 있는데 즐거워
4,스팀에서 재밌는 거 해야지
5,스팀에어프라이어로 연어구이 해먹을거야


In [6]:
def get_embedding(text):
  return embeddings.embed_query(text)

df['embedding'] = df.apply(lambda row: get_embedding(
        row.text,
    ), axis=1)
df

,text,embedding
0,저는 배가 고파요,"[-0.016637360179694066, -0.021788898203996467,..."
1,저기 배가 지나가네요,"[-0.0032914344795325046, -0.02751476582446401,..."
2,굶어서 허기가 지네요,"[-0.006181030746934311, -0.006950793503402665,..."
3,허기 워기라는 게임이 있는데 즐거워,"[-0.011329255872108562, -0.01171585252514384, ..."
4,스팀에서 재밌는 거 해야지,"[-0.016108456687859818, -0.014401600417918037,..."
5,스팀에어프라이어로 연어구이 해먹을거야,"[-0.0021389091187001565, -0.03003427992224648,..."


In [7]:
def cos_sim(A, B):
  return dot(A, B)/(norm(A)*norm(B))

def return_answer_candidate(df, query):
    query_embedding = get_embedding(
        query
    )
    df["similarity"] = df.embedding.apply(lambda x: cos_sim(np.array(x),
                                                            np.array(query_embedding)))
    top_three_doc = df.sort_values("similarity",
                                ascending=False).head(3)
    return top_three_doc

In [8]:
sim_result = return_answer_candidate(df, '아무것도 안 먹었더니 꼬르륵 소리가 나네')
sim_result

,text,embedding,similarity
2,굶어서 허기가 지네요,"[-0.006181030746934311, -0.006950793503402665,...",0.836768
5,스팀에어프라이어로 연어구이 해먹을거야,"[-0.0021389091187001565, -0.03003427992224648,...",0.815944
0,저는 배가 고파요,"[-0.016637360179694066, -0.021788898203996467,...",0.812184
